In [1]:
%%writefile conda_dependencies.yml

channels:
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-defaults
  - tensorflow-gpu==2.2.0
  - pandas

Writing conda_dependencies.yml


In [6]:
#!pip install azureml-contrib-fairness
#!pip install fairlearn==0.4.6
#!pip install raiwidgets
#pip install azureml-dataprep[pandas]

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Environment
from azureml.core import *
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration
from azureml.core import Workspace, Dataset, Datastore
import os, shutil
from azureml.core import Dataset
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.core import Workspace, Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace,RunConfiguration
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep
from azureml.train.automl.utilities import get_primary_metrics
from azureml.data.datapath import DataPath
from azureml.core.datastore import Datastore
from azureml.pipeline.core import InputPortBinding
from azureml.core import Run
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.pipeline.core import PipelineParameter
from azureml.core import Dataset
from azureml.core import Environment
from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.pipeline.steps import HyperDriveStep, HyperDriveStepRun, PythonScriptStep
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.data import OutputFileDatasetConfig
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.hyperdrive import *
from azureml.widgets import RunDetails
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule
from azureml.core import Run
from azureml.core import get_run
import pandas as pd
import numpy as np
from fairlearn.reductions import GridSearch
from fairlearn.reductions import DemographicParity
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from interpret.ext.blackbox import MimicExplainer
from interpret.ext.glassbox import LGBMExplainableModel
from raiwidgets import FairnessDashboard
from raiwidgets import ExplanationDashboard
from azureml.pipeline.core import Pipeline
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies


# connection au WS
from azureml.core import Workspace
ws = Workspace.get(subscription_id = "---------------",
                   resource_group = "------------",
                   name = "------------")


# Execution du script en tant qu'experience
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.pipeline.core import Pipeline
from azureml.data import OutputFileDatasetConfig

# Create a Python environment for the experiment
#sklearn_env = Environment("sklearn-env")
experiment = Experiment(ws, 'Firstpipeline') 

from azureml.core import Environment
from azureml.core.runconfig import RunConfiguration
env = Environment.from_conda_specification(name='my-env', file_path='./conda_dependencies.yml')
env.docker.enabled = True
env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04'
# create a new runconfig object
runconfig = RunConfiguration()
runconfig.environment = env



dataprep_step = PythonScriptStep(
    name="prep_data",
    script_name="scripts/datacleaning_scipt1.py",
    source_directory=".",
    compute_target='jucalcul1',
    #arguments=["--prepped_data_path", prepped_data_path],
    #inputs=[input_dataset.as_named_input('raw_data').as_mount() ]
    )

OutputFileDatasetConfig(name="output_path")

train_step = PythonScriptStep(
    name="train",
    script_name="scripts/model_script2.py",
    compute_target='jucalcul1',
    #arguments=["--prepped_data", prepped_data],
    source_directory="."
)
steps = [dataprep_step, train_step ]


pipeline = Pipeline(workspace=ws, steps=steps)

pipeline_run = experiment.submit(pipeline)
pipeline_run.wait_for_completion()

'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


Created step prep_data [827a0da0][afdf9cef-fcdb-48ec-9ea0-a3d57b68ea8b], (This step will run and generate new outputs)
Created step train [311404a3][d63611b9-69ff-42e7-be5e-f1254e094801], (This step will run and generate new outputs)
Submitted PipelineRun 433a1ae1-628c-40c6-aaec-f60ffe3ce5c7
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/433a1ae1-628c-40c6-aaec-f60ffe3ce5c7?wsid=/subscriptions/a0f4cddc-a66a-4dcc-9df7-ccbd7f81bf7b/resourcegroups/ressourcesecu/workspaces/deuxiemeespace&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
PipelineRunId: 433a1ae1-628c-40c6-aaec-f60ffe3ce5c7
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/433a1ae1-628c-40c6-aaec-f60ffe3ce5c7?wsid=/subscriptions/a0f4cddc-a66a-4dcc-9df7-ccbd7f81bf7b/resourcegroups/ressourcesecu/workspaces/deuxiemeespace&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
PipelineRun Status: Running


Expected a StepRun object but received <class 'azureml.core.run.Run'> instead.
This usually indicates a package conflict with one of the dependencies of azureml-core or azureml-pipeline-core.
Please check for package conflicts in your python environment
